In [1]:
import os
os.chdir("../")

In [2]:
from dataclasses import dataclass
from pathlib import Path


In [3]:
@dataclass(frozen=True)
class TrainingConfig:
    root_dir: Path
    trained_model_path: Path
    updated_base_model_path: Path
    training_data: Path
    params_epochs: int
    params_is_augmentation: bool
    params_image_size: list
    params_batch_size: int

In [4]:
@dataclass(frozen=True)
class PrepareCallbackConfig:
    root_dir: Path
    tensorboard_root_log_dir: Path
    checkpoint_model_filepath: Path

In [5]:
from deepclassifier.constants import *
from deepclassifier.utils import read_yaml, create_directories
import tensorflow as tf

In [6]:
class ConfigurationManager:
    def __init__(self,
        config_filepath=CONFIG_FILE_PATH,
        params_filepath=PARAMS_FILE_PATH):
        
        self.config=read_yaml(config_filepath)
        self.params=read_yaml(params_filepath)
        create_directories([self.config.artifacts_root])
        
    def get_prepare_callback_config(self)-> PrepareCallbackConfig:
        config=self.config.prepare_callbacks
        model_ckpt_dir=os.path.dirname(config.checkpoint_model_filepath)
        create_directories([
            Path(model_ckpt_dir),
            Path(config.tensorboard_root_log_dir)
        ])

        prepare_callback_config=PrepareCallbackConfig(
            root_dir=Path(config.root_dir),
            tensorboard_root_log_dir=Path(config.tensorboard_root_log_dir),
            checkpoint_model_filepath=Path(config.checkpoint_model_filepath)
        )

        return prepare_callback_config

    def get_training_config(self)-> TrainingConfig:
        training=self.config.training
        prepare_base_model=self.config.prepare_base_model
        params=self.params
        training_data=os.path.join(self.config.data_ingestion.unzip_dir,"PetImages")
        create_directories([Path(training.root_dir)])
        print(params)
        
        training_config = TrainingConfig(
            root_dir=Path(training.root_dir),
            trained_model_path=Path(training.trained_model_path),
            updated_base_model_path=Path(prepare_base_model.updated_base_model_path),
            training_data=Path(training_data),
            params_epochs=params.EPOCHS,
            params_batch_size=params.BATCH_SIZE,
            params_is_augmentation=params.AUGMENTATION,
            params_image_size=params.IMAGE_SIZE
        )

        return training_config


In [7]:
import time

class PrepareCallback:
    def __init__(self, config: PrepareCallbackConfig):
        self.config = config

    @property
    def _create_tb_callbacks(self):
        timestamp = time.strftime("%Y-%m-%d-%H-%M-%S")
        tb_running_log_dir = os.path.join(
            self.config.tensorboard_root_log_dir,
            f"tb_logs_at_{timestamp}",
        )
        return tf.keras.callbacks.TensorBoard(log_dir=tb_running_log_dir)

    @property
    def _create_ckpt_callbacks(self):
        return tf.keras.callbacks.ModelCheckpoint(
            filepath=self.config.checkpoint_model_filepath,
            save_best_only=True
        )

    def get_tb_ckpt_callbacks(self):
        return [
            self._create_tb_callbacks,
            self._create_ckpt_callbacks
        ]

In [20]:
import os
import tensorflow as tf

class Training:
    def __init__(self,config: TrainingConfig):
        self.config=config
    
    def get_base_model(self):
        self.model=tf.keras.models.load_model(
            self.config.updated_base_model_path
        )
    def train_validation_generator(self):
        
        datagenrator_kwargs=dict(
            rescale=1./255,
            validation_split=0.20
        )

        dataflow_kwargs=dict(
            target_size=self.config.params_image_size[:-1], #Not accepting the channel size
            batch_size=self.config.params_batch_size,
             interpolation="bilinear"
            )
        
        valid_datagenrator=tf.keras.preprocessing.image.ImageDataGenerator(
            **datagenrator_kwargs
        )
        self.valid_datagenator=valid_datagenrator.flow_from_directory(
            directory=self.config.training_data,
            subset="validation",
            shuffle=False,
            **dataflow_kwargs
        )

        if self.config.params_is_augmentation:
            train_datagenerator = tf.keras.preprocessing.image.ImageDataGenerator(
                rotation_range=40,
                horizontal_flip=True,
                width_shift_range=0.2,
                height_shift_range=0.2,
                shear_range=0.2,
                zoom_range=0.2,
                **datagenrator_kwargs
            )

        else:
            train_datagenerator = valid_datagenrator

        self.train_generator=train_datagenerator.flow_from_directory(
            directory=self.config.training_data,
            subset="training",
            shuffle=True,
            **dataflow_kwargs
        )

        

    def train(self,callbacklist: list):
        self.steps_per_epoch = self.train_generator.samples//self.train_generator.batch_size
        self.validation_steps=self.valid_datagenator.samples//self.valid_datagenator.batch_size

        self.model.fit(
            self.train_generator,
            steps_per_epoch=self.steps_per_epoch,
            epochs=self.config.params_epochs,
            validation_steps=self.validation_steps,
            validation_data=self.valid_datagenator,
            callbacks=callbacklist,

        )
        self.save_model(
        path=self.config.trained_model_path,
        model=self.model
    )
    
    @staticmethod
    def save_model(path:Path,model:tf.keras.Model):
        model.save(path)


In [21]:
try:
    config = ConfigurationManager()
    prepare_callbacks_config = config.get_prepare_callback_config()
    prepare_callbacks = PrepareCallback(config=prepare_callbacks_config)
    callback_list = prepare_callbacks.get_tb_ckpt_callbacks()
    
    training_config = config.get_training_config()
    training = Training(config=training_config)
    training.get_base_model()
    training.train_validation_generator()
    training.train(
        callbacklist=callback_list
    )
    
except Exception as e:
    raise e

[2022-11-10 21:12:08,231: INFO: common]: yaml file: configs\config.yaml loaded successfully
[2022-11-10 21:12:08,234: INFO: common]: yaml file: params.yaml loaded successfully
[2022-11-10 21:12:08,235: INFO: common]: created directory at: artifacts
[2022-11-10 21:12:08,238: INFO: common]: created directory at: artifacts\prepare_callbacks\checkpoint_dir
[2022-11-10 21:12:08,242: INFO: common]: created directory at: artifacts\prepare_callbacks\tensorboard_log_dir
[2022-11-10 21:12:08,245: INFO: common]: created directory at: artifacts\training
{'AUGMENTATION': True, 'IMAGE_SIZE': [224, 224, 3], 'BATCH_SIZE': 16, 'INCLUDE_TOP': False, 'EPOCHS': 1, 'CLASSES': 2, 'WEIGHTS': 'imagenet', 'LEARNING_RATE': 0.01}
Found 4998 images belonging to 2 classes.
Found 20000 images belonging to 2 classes.
 736/1250 [================>.............] - ETA: 17:18 - loss: 7.8644 - accuracy: 0.6472

d:\anaconda3\envs\deepCNNApp\lib\site-packages\PIL\TiffImagePlugin.py:850: UserWarning: Truncated File Read
  warnings.warn(str(msg))


1250/1250 [==============================] - 2921s 2s/step - loss: 6.3611 - accuracy: 0.6859 - val_loss: 1.7964 - val_accuracy: 0.8588


In [23]:
def example_for_understanding(x,**kwargs):
    print(locals())

In [25]:
kwargs = {"arg1": "Geeks", "arg2": "for", "arg3": "Geeks"}

extra1=example_for_understanding(5,**kwargs)

{'x': 5, 'kwargs': {'arg1': 'Geeks', 'arg2': 'for', 'arg3': 'Geeks'}}


In [26]:
extra1=example_for_understanding(5,arg1='Geeks', arg2= 'for', arg3= 'Geeks')

{'x': 5, 'kwargs': {'arg1': 'Geeks', 'arg2': 'for', 'arg3': 'Geeks'}}


In [31]:
kwargs2=dict(arg1='Geeks', arg2= 'for', arg3= 'Geeks')
kwargs2

{'arg1': 'Geeks', 'arg2': 'for', 'arg3': 'Geeks'}

In [32]:
extra2=example_for_understanding(10,**kwargs2)

{'x': 10, 'kwargs': {'arg1': 'Geeks', 'arg2': 'for', 'arg3': 'Geeks'}}
